# <h1><center>Demo 5</center></h1>

## Table Join vs. Spatial join


In [1]:
# Import Geopandas and matplotlib
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [9]:
# Read Countries data
shp = 'data/countries.shp'
countries = gpd.read_file(shp)
countries.head()

,NAME,POP_EST,GDP_MD_EST,POP_YEAR,LASTCENSUS,GDP_YEAR,ECONOMY,INCOME_GRP,CONTINENT,REGION_UN,SUBREGION,geometry
0,Indonesia,260580739,3028000.0,2017,2010,2016,4. Emerging region: MIKT,4. Lower middle income,Asia,Asia,South-Eastern Asia,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Malaysia,31381992,863000.0,2017,2010,2016,6. Developing region,3. Upper middle income,Asia,Asia,South-Eastern Asia,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Chile,17789267,436100.0,2017,2002,2016,5. Emerging region: G20,3. Upper middle income,South America,Americas,South America,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Bolivia,11138234,78350.0,2017,2001,2016,5. Emerging region: G20,4. Lower middle income,South America,Americas,South America,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Peru,31036656,410400.0,2017,2007,2016,5. Emerging region: G20,3. Upper middle income,South America,Americas,South America,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


In [10]:
# Read Conflict data
conflict_shp = 'data/ConflictData.shp'
conflict = gpd.read_file(conflict_shp)
conflict.head()

,data_id,iso,event_id_c,event_id_n,event_date,year,time_preci,event_type,sub_event_,actor1,...,latitude,longitude,geo_precis,source,source_sca,notes,fatalities,timestamp,iso3,geometry
0,6768128,887,YEM45982,45982,31 December 2019,2019,1,Battles,Armed clash,Military Forces of Yemen (2012-),...,14.3541,47.0765,2,Yemen Data Project; Aden al Ghad; Al Janoob al...,Local partner-National,"On 31 December 2019, clashes reportedly erupte...",0,1580761030,YEM,POINT (47.07650 14.35410)
1,6768129,887,YEM45983,45983,31 December 2019,2019,1,Strategic developments,Change to group/activity,Military Forces of Yemen (2012-) Security Belt...,...,14.3541,47.0765,2,Al Janoob al Youm; Yemen Data Project,Local partner-National,"On 31 December 2019, forces loyal to the South...",0,1580761030,YEM,POINT (47.07650 14.35410)
2,6768130,887,YEM45984,45984,31 December 2019,2019,1,Battles,Armed clash,Military Forces of Yemen (2012-),...,14.2081,47.1570,1,Al Janoob al Youm; Aden al Ghad; Yemen Data Pr...,Local partner-National,"On 31 December 2019, clashes reportedly erupte...",1,1580761030,YEM,POINT (47.15700 14.20810)
3,6715395,356,IND68948,68948,31 December 2019,2019,1,Protests,Peaceful protest,Protesters (India),...,20.2724,85.8338,1,Pioneer (India),National,"On 31 December, students' wing of INC and Mili...",0,1578514389,IND,POINT (85.83380 20.27240)
4,6943236,222,SLV861,861,31 December 2019,2019,1,Violence against civilians,Attack,B-18: Barrio-18,...,13.5894,-89.2383,1,El Salvador.com,National,On 31th December 2019 in Huizucar (La Libertad...,1,1582839958,SLV,POINT (-89.23830 13.58940)


### Table Join

In [4]:
# To simplify let us seperate country table into two tables. 

# This table holds the geometry and abbreviated names 

country_shapes = countries[['geometry', 'NAME']]
country_shapes.head()

,geometry,NAME
0,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...",Indonesia
1,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...",Malaysia
2,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...",Chile
3,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...",Bolivia
4,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...",Peru


In [5]:
type(country_shapes)

geopandas.geodataframe.GeoDataFrame

Notice that it is a geodataframe as it has a geometry column.

In [11]:
# This table holds Country names and population estimate
country_names = countries[['NAME', 'POP_EST']]
country_names.head()

,NAME,POP_EST
0,Indonesia,260580739
1,Malaysia,31381992
2,Chile,17789267
3,Bolivia,11138234
4,Peru,31036656


In [12]:
type(country_names)

pandas.core.frame.DataFrame

Notice that it is not a geodataframe as it does not have any geometry column.

**We can join/merge the two tables based on their shared column NAME. This is pure pandas operation and does not entail any geographic operations**

In [13]:
# Merge country_shapes to country_names on NAME Column. 
country_gdf_merged = country_shapes.merge(country_names, on='NAME', how='left')
country_gdf_merged.head()

,geometry,NAME,POP_EST
0,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...",Indonesia,260580739
1,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...",Malaysia,31381992
2,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...",Chile,17789267
3,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...",Bolivia,11138234
4,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...",Peru,31036656


In [14]:
# Merge country_shapes to country names on NAME Column. 
country_df_merged = country_names.merge(country_shapes, on='NAME', how='left')
country_df_merged.head()

,NAME,POP_EST,geometry
0,Indonesia,260580739,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Malaysia,31381992,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Chile,17789267,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Bolivia,11138234,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Peru,31036656,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


The two tables are merged above. This is called Table join and does not involve any geographic relations. It involves only table attributes. However, we can take benefit of Geodataframe options to do Spatial Join, in that case, we merge two geometry objects based on their locations. Let us see an example.

### Spatial Join

In [15]:
country_shapes.head()

,geometry,NAME
0,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...",Indonesia
1,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...",Malaysia
2,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...",Chile
3,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...",Bolivia
4,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...",Peru


In [17]:
conflict_gdf = conflict[['event_id_c', 'event_date','fatalities', 'geometry']]
conflict_gdf.head()

,event_id_c,event_date,fatalities,geometry
0,YEM45982,31 December 2019,0,POINT (47.07650 14.35410)
1,YEM45983,31 December 2019,0,POINT (47.07650 14.35410)
2,YEM45984,31 December 2019,1,POINT (47.15700 14.20810)
3,IND68948,31 December 2019,0,POINT (85.83380 20.27240)
4,SLV861,31 December 2019,1,POINT (-89.23830 13.58940)



- We will perform an example of a spatial join. 
- We want to join the two tables based on their locations. 
- As per the ConflicData.shp data we know the point coordinate for each incidents but we have no idea about the country under which a particular incident occured. At least not by simply looking at the GeoDataFrame or table. 
- Spatial join can be helpful in this kind of situations using the Geopandas function, 
```
sjoin()

```

to do the spatial join.


In [18]:
# Merge conflicts with country names and population
conflicts_in_countries = gpd.sjoin(conflict_gdf, country_shapes, how="inner", op='within')
conflicts_in_countries.sample(10)

/Users/nonso/code/jupyter/geospatial/env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/3x/5jj60pms5ml09xsn32f0c4yr0000gn/T/ipykernel_59797/3258009935.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84" ...

  conflicts_in_countries = gpd.sjoin(conflict_gdf, country_shapes, how="inner", op='within')


,event_id_c,event_date,fatalities,geometry,index_right,NAME
156081,TUR7653,21 March 2019,0,POINT (44.08420 39.54690),66,Turkey
170376,XKX185,23 February 2019,0,POINT (21.35950 42.32230),65,Kosovo
113672,BFO1994,09 June 2019,2,POINT (-0.07240 14.56190),148,Burkina Faso
38033,HTI321,25 October 2019,0,POINT (-73.08760 18.44230),112,Haiti
9780,PAK55068,12 December 2019,0,POINT (74.65570 32.25890),17,Pakistan
163556,IND47690,07 March 2019,2,POINT (74.86910 32.73570),8,India
127491,IRN4232,15 May 2019,0,POINT (51.42610 35.76610),93,Iran
27632,SYR70338,11 November 2019,0,POINT (36.37900 35.54990),20,Syria
65259,DRC15487,08 September 2019,2,POINT (28.19830 -3.04720),32,Dem. Rep. Congo
134465,YEM31095,02 May 2019,8,POINT (44.70580 13.85490),161,Yemen


Each conflict point is matched with its corresponding country based on the location. We have used op=within which takes conflict point that are within a countries polygon.